In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing dataset from kaggle

In [3]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 95% 77.0M/80.9M [00:03<00:00, 17.5MB/s]
100% 80.9M/80.9M [00:03<00:00, 25.8MB/s]


Extraction the zip file

In [4]:
from zipfile import ZipFile
dataset = '/content/drive/MyDrive/Colab Notebooks/Twitter_sentiment_analysis/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [40]:
import numpy as np
import pandas as pd
import re #Regular Expression(pattern matching, searching through the data)
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00


In [6]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
twitter_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Twitter_sentiment_analysis/twitter_dataset.1600000.processed.neomoticon.csv',encoding='ISO-8859-1')

In [8]:
twitter_dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [9]:
#naming the colums
column_names = ["target","ids","date","flag","user","text"]
twitter_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Twitter_sentiment_analysis/twitter_dataset.1600000.processed.neomoticon.csv',encoding='ISO-8859-1',names=column_names)
twitter_dataset.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
twitter_dataset.shape

(1600000, 6)

In [11]:
twitter_dataset.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [12]:
twitter_dataset['target'].value_counts()

,count
target,
0,800000
4,800000


In [13]:
twitter_dataset['target'].replace(4, 1, inplace=True)

<ipython-input-13-ed6d26f6b4a0>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  twitter_dataset['target'].replace(4, 1, inplace=True)


In [14]:
twitter_dataset['target'].value_counts()

,count
target,
0,800000
1,800000


Stemming

In [15]:
port_stem = PorterStemmer()

In [16]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #removing all except a to z and A to Z
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [17]:
twitter_dataset['stemmed_content'] = twitter_dataset['text'].apply(stemming)

In [18]:
twitter_dataset.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [28]:
x = twitter_dataset['stemmed_content'].values
y = twitter_dataset['target'].values

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2) #stratify= the same proportion of 0s and 1s in training data as well as testing data

In [51]:
vectorizor = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)
x_train_vectorised = vectorizor.fit_transform(x_train)
x_test_vectorised = vectorizor.transform(x_test)

In [52]:
model = LogisticRegression(max_iter=2000)

In [53]:
model.fit(x_train_vectorised, y_train)

LogisticRegression(max_iter=2000)

Model Evaluation

In [54]:
#accuracy score on the training data
x_train_prediction = model.predict(x_train_vectorised)
training_data_accuracy = accuracy_score(y_train, x_train_prediction)
print("The accuracy score on the training data: ", training_data_accuracy)

The accuracy score on the training data:  0.79870546875


In [55]:
#accuracy score on the testing data
x_test_prediction = model.predict(x_test_vectorised)
testing_data_accuracy = accuracy_score(y_test, x_test_prediction)
print("The accuracy score on the testing data: ", testing_data_accuracy)

The accuracy score on the testing data:  0.772240625


In [56]:
import pickle
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [57]:
filename2 = 'Vectorizor.sav'
pickle.dump(vectorizor, open(filename2, 'wb'))

In [74]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import pickle


#Title
st.title('Twitter Sentiment Analysis')

#loading Model
model = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Twitter_sentiment_analysis/trained_model.sav', 'rb'))
vectorizor = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Twitter_sentiment_analysis/Vectorizor.sav', 'rb'))

#Input
tweet = st.text_input(
    'Enter the Tweet:- ',
    ""
)

def prediction(tweet):
  vectorised_tweet = vectorizor.transform([tweet])
  prediction = model.predict(vectorised_tweet)

  if prediction[0] == 1:
    return "Positive Tweet"
  else:
    return "Negative Tweet"


if st.button('Predict'):
  result = prediction(tweet)
  if result == "Positive Tweet":
    st.success("Positive Tweet")
  else:
    st.error("Negative Tweet")

Overwriting app.py


In [75]:
!wget -q -O - ipv4.icanhazip.com # copy the code below and paste it in the 'your url is:' generated in the below cell

34.91.123.69


In [84]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.123.69:8501

your url is: https://true-coats-decide.loca.lt
  Stopping...
^C
